In [73]:
using Oscar

In [74]:
R, (x, y, z) = PolynomialRing(QQ, ["x","y","z"],ordering = :degrevlex)


(Multivariate Polynomial Ring in x, y, z over Rational Field, fmpq_mpoly[x, y, z])

In [75]:
f1 = x^2 - 2 * x * z + 5
f2 = x * y^2 + y * z + 1
f3 = 3 * y^2 - 8 * x * z
I = ideal(R,[f1,f2,f3])

ideal(x^2 - 2*x*z + 5, x*y^2 + y*z + 1, 3*y^2 - 8*x*z)

In [76]:
G = groebner_basis(I)


Gröbner basis with elements
1 -> 3*y^2 - 8*x*z
2 -> x^2 - 2*x*z + 5
3 -> 160*z^3 - 160*x*z + 415*y*z + 12*x - 30*y - 224*z + 15
4 -> 240*y*z^2 - 9*x*y + 1600*x*z + 18*y*z + 120*z^2 - 120*x + 240*z
5 -> 16*x*z^2 + 3*y*z - 40*z + 3
6 -> 3*x*y*z - 6*y*z^2 - 40*x*z + 3*x - 6*z
with respect to the ordering
degrevlex([x, y, z])

In [77]:
Gi = ideal(G) 

ideal(3*y^2 - 8*x*z, x^2 - 2*x*z + 5, 160*z^3 - 160*x*z + 415*y*z + 12*x - 30*y - 224*z + 15, 240*y*z^2 - 9*x*y + 1600*x*z + 18*y*z + 120*z^2 - 120*x + 240*z, 16*x*z^2 + 3*y*z - 40*z + 3, 3*x*y*z - 6*y*z^2 - 40*x*z + 3*x - 6*z)

In [78]:
function dimQI(v,lista)
    max = 0
    leadmons = Set([leading_monomial(f) for f in lista])
    for p in leadmons
        if v in vars(p)
            m = degree(p, v)
            if m> max
                max = m
            end
        end
    end
    return max
end

dimQI (generic function with 1 method)

In [79]:
function kbasis(GB, VList)
    # devuelve una lista de monomios que forman una base del anillo cociente, donde GB es una base de Gröbner para un ideal de dimensión cero, y genera un mensaje de error si el ideal no es de dimensión cero.
    
    if iszero(dim(GB))  # verifica si el ideal es de dimensión cero
        Gb = collect(gens(GB))
        B = [(div(VList[1],VList[1])) ]  # inicializa la lista de monomios de la base con el monomio 1
        leadmons = Set([leading_monomial(f) for f in Gb])  # crea un conjunto de los monomios principales de cada polinomio en la base de Gröbner
        for v in VList
            m = dimQI(v, leadmons)  # determina el grado de la variable v en el anillo cociente
            C = copy(B)  # crea una copia de la lista de monomios actual como base para la siguiente iteración
            for t in C
                for l in 1:m-1
                    t = t*v  # multiplica el monomio actual por la variable v
                    if  !(any((divrem(t,u)[2] == 0) for u in leadmons))  # verifica si alguno de los monomios principales de la base de Gröbner divide al nuevo monomio
                        push!(B, t)  # si ningún monomio principal lo divide, agrega el nuevo monomio a la lista de monomios de la base
                    end
                end
            end
        end
        return B  # devuelve la lista de monomios de la base
    else
        error("el ideal no es de dimensión cero")
    end
end

kbasis (generic function with 1 method)

In [80]:
KB = kbasis(Gi,[x,y,z])

8-element Vector{fmpq_mpoly}:
 1
 x
 y
 x*y
 z
 z^2
 x*z
 y*z

In [81]:
kb = ideal(R,KB)

ideal(1, x, y, x*y, z, z^2, x*z, y*z)

In [82]:
function matrixM(variable,kbase,Gbase)
    kbx = kbase*variable
    reducida = reduce(collect(gens(kbx)),collect(Gbase))
    Mx = [coeff(kr,k) for k in collect(gens(kbase)) , kr in collect(reducida)] 
    return Mx
end

matrixM (generic function with 1 method)

In [83]:
matrixM(x,kb,G)

8×8 Matrix{fmpq}:
 0  -5  0  0       0  -3//16  -3//8  0
 1  0   0  0       0  0       0      0
 0  0   0  -5      0  0       0      0
 0  0   1  3//20   0  0       0      3//40
 0  0   0  0       0  5//2    0      0
 0  0   0  -2      0  0       0      -1
 0  2   0  0       1  0       0      0
 0  0   0  -3//10  0  -3//16  -3//8  -3//20

In [84]:
matrixM(y,kb,G)

8×8 Matrix{fmpq}:
 0  0  0     -1  0  0       0       -1//2
 0  0  0     0   0  1//2    0       0
 1  0  0     0   0  0       0       0
 0  1  0     0   0  3//80   3//40   0
 0  0  0     0   0  -1      0       20//3
 0  0  0     0   0  -1//2   -1      0
 0  0  8//3  0   0  -20//3  0       0
 0  0  0     -1  1  -3//40  -3//20  -1//2

In [85]:
matrixM(z,kb,G)

8×8 Matrix{fmpq}:
 0  0  0  0       0  -3//32   -3//16  0
 0  0  0  0       0  -3//40   0       1//2
 0  0  0  0       0  3//16    0       0
 0  0  0  3//40   0  0        0       3//80
 1  0  0  0       0  7//5     5//2    -1
 0  0  0  -1      1  0        0       -1//2
 0  1  0  0       0  1        0       -20//3
 0  0  1  -3//20  0  -83//32  -3//16  -3//40

In [86]:

R2, (x, y) = PolynomialRing(QQ, ["x","y"],ordering = :degrevlex)
f1 = x^2 + (3//2)*x*y + (1//2)*y^2  - (3//2)*x - (3//2)*y
f2 = x*y^2 - x
f3 = y^3 - y
I2 = ideal(R2,[f1,f2,f3])
G2 = groebner_basis(I2)
Gi2 = ideal(G2) 


ideal(2*x^2 + 3*x*y + y^2 - 3*x - 3*y, y^3 - y, x*y^2 - x)

In [87]:
KB2 = kbasis(Gi2,[x,y])
kb2 = ideal(R2,KB2)

ideal(1, x, y, y^2, x*y)

In [88]:

matrixM(x,kb2,G2)


5×5 Matrix{fmpq}:
 0  0      0  0  0
 1  3//2   0  1  -3//2
 0  3//2   0  0  -1//2
 0  -1//2  0  0  3//2
 0  -3//2  1  0  3//2